In [1]:
from fol import evaluate
from mlx_lm import load, generate 
from datasets import load_dataset
from folio_dataset import create_prompt, instruction, get_example_prompt_str, fol_to_nltk

folio_train = load_dataset('yale-nlp/FOLIO', split='train')
folio_val = load_dataset('yale-nlp/FOLIO', split='validation')

folio_train = folio_train.map(fol_to_nltk)
folio_val = folio_val.map(fol_to_nltk)

example_prompt_str = get_example_prompt_str(folio_train, n_shots=4)

folio_val = folio_val.map(
    create_prompt,
    fn_kwargs=dict(
        instruction=instruction,
        example_prompt_str=example_prompt_str
    )
)

/Users/zhijiezhu/.pyenv/versions/3.11.8/envs/mlx/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Map: 100%|██████████| 203/203 [00:00<00:00, 12720.08 examples/s]


In [2]:
print(folio_val[0]['prompt'])

The following is a first-order logic (FOL) problem.
The problem is to determine whether the conclusion follows from the premises.
The premises are given in the form of a set of first-order logic sentences.
The conclusion is given in the form of a single first-order logic sentence.
The task is to translate each of the premises and conclusions into FOL expressions,so that the expressions can be evaluated by a theorem solver to determine whether the conclusion follows from the premises.
Expressions should be adhere to the format of the provided examples.

<PREMISES>
A summarization model is always faithful if it uses content from the input documents.
Extractive models are summarization models.
An extractive model can only use content from the input documents.
<PREMISES>
<CONCLUSION>
Extractive models are not always faithful.
<CONCLUSION>
<EVALUATE>
TEXT: A summarization model is always faithful if it uses content from the input documents.
FOL: all x. (Model(x) & Summarization(x) & OnlyUse

In [3]:
model, tokenizer = load("mlx-community/quantized-gemma-7b-it")
response = generate(model, tokenizer, prompt=folio_val[0]['prompt'], verbose=True, max_tokens=1024)

Fetching 8 files: 100%|██████████| 8/8 [00:00<00:00, 82443.32it/s]


Prompt: The following is a first-order logic (FOL) problem.
The problem is to determine whether the conclusion follows from the premises.
The premises are given in the form of a set of first-order logic sentences.
The conclusion is given in the form of a single first-order logic sentence.
The task is to translate each of the premises and conclusions into FOL expressions,so that the expressions can be evaluated by a theorem solver to determine whether the conclusion follows from the premises.
Expressions should be adhere to the format of the provided examples.

<PREMISES>
A summarization model is always faithful if it uses content from the input documents.
Extractive models are summarization models.
An extractive model can only use content from the input documents.
<PREMISES>
<CONCLUSION>
Extractive models are not always faithful.
<CONCLUSION>
<EVALUATE>
TEXT: A summarization model is always faithful if it uses content from the input documents.
FOL: all x. (Model(x) & Summarization(x) &

In [11]:
llm_translated_fols = []
for line in response.splitlines():
    if not line.startswith('FOL'):
        continue
    llm_translated_fols.append(line[5:])

evaluate(llm_translated_fols[:-1], llm_translated_fols[-1])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'Uncertain'